In [107]:
import numpy as np
import pandas as pd
from matplotlib import pyplot
print(np.__version__)
#from sklearn.metrics import ConfusionMatrixDisplay
#import seaborn

import os
import sys
if '../../lib/' not in sys.path:
    sys.path.append('../../lib/')

import multi_fgl_utils as util
import plot_util as putil
import dio3

1.26.4


In [108]:
if 1:
    import importlib
    importlib.reload(util);


In [109]:
if 0:
    save_plots = 1
    save_png = 1

    script_fn = 'GU_git/notebooks/ML/joint_cov_prior_prediction.ipynb'
    fig_meta = {'Author': 'Dmitry Malyshev', 'Subject': script_fn}

    plots_folder = '../../plots/ML/'
    if not os.path.isdir(plots_folder):
        print('Make folder')
        os.makedirs(plots_folder)
    print(plots_folder)


In [110]:
# load catalogs and create output dataframe
data_folder = '../../data_in/ML_catalogs/'
fn_out = data_folder + '4FGL-DR4_v34_cov_shift_vs_prior_shift_prediction.csv'
fn = data_folder + '4FGL-DR4_v34_4classes_GMM_no_bcu_spp_no_coord_features_prob_cat.csv'
summary_fn = fn.replace('prob_cat.csv', 'summary.csv')
cat_df1 = pd.read_csv(fn, index_col=0)
sum_df1 = pd.read_csv(summary_fn, index_col=1, dtype=str)
print(fn)
print(len(cat_df1))

fn = data_folder + '4FGL-DR4_v34_4classes_GMM_no_bcu_spp_no_coord_features_unas_unk_prior_shift_npar4_1Gauss_prob_cat.csv'
summary_fn = fn.replace('prob_cat.csv', 'summary.csv')
cat_df2 = pd.read_csv(fn, index_col=0)
sum_df2 = pd.read_csv(summary_fn, index_col=1, dtype=str)
gind = sum_df2['Group'].loc['Gauss']
cat_df2['%s_PDF' % gind] = cat_df2['Gauss_PDF']
print(fn)
print(len(cat_df2))

# subselect sources in cat_df1 which are also in cat_df2
cat_df1 = cat_df1.loc[cat_df2.index]
print(len(cat_df1), len(cat_df2))
classes = sum_df1.index
for cl in sum_df1.index:
    print('%s: %s' %(cl, sum_df1['Physical classes'].loc[cl]))

cat_dfs = [cat_df1, cat_df2]
sum_dfs = [sum_df1, sum_df2]
methods = ['RF', 'PDF']

add_unc = True
unc_name = 'uncert'
pmin = 0.5
out_df = cat_df2.copy()
for i in range(1, len(classes) + 1):
    out_df.pop('%i_PDF' % i)
gauss_probs = out_df.pop('Gauss_PDF')

../../data_in/ML_catalogs/4FGL-DR4_v34_4classes_GMM_no_bcu_spp_no_coord_features_prob_cat.csv
7191
../../data_in/ML_catalogs/4FGL-DR4_v34_4classes_GMM_no_bcu_spp_no_coord_features_unas_unk_prior_shift_npar4_1Gauss_prob_cat.csv
5859
5859 5859
fsrq+: fsrq, nlsy1, css
bll+: bll, sey, sbg, agn, ssrq, rdg
psr+: snr, hmb, nov, pwn, psr, gc
msp+: msp, lmb, glc, gal, sfr, bin


In [111]:
if 0:
    print(classes)
    print(cat_df1.columns)
    print(cat_df2.columns)

In [112]:
# find maximal probabilities and corresponding classes
for k in range(2):
    classes = sum_dfs[k].index
    prob_clms = ['%i_%s' % (i, methods[k]) for i in range(1, len(classes) + 1)]
    probs = np.array(cat_dfs[k][prob_clms])
    out_df['%s_prob' % methods[k]] = np.max(probs, axis=1) 
    out_df['%s_class' % methods[k]] = classes[np.argmax(probs, axis=1)]
    if add_unc and 0:
        unc_mask = np.max(probs, axis=1) < pmin
        out_df.loc[unc_mask, '%s_class' % methods[k]] = unc_name

In [113]:
#cat_df2

In [114]:
save_results = True
if save_results:
    print('Save the result to file:')
    print(fn_out)
    out_df.to_csv(fn_out)

Save the result to file:
../../data_in/ML_catalogs/4FGL-DR4_v34_cov_shift_vs_prior_shift_prediction.csv


In [115]:
out_df

,log10(Energy_Flux100),LP_beta,log10(LP_EPeak),GLON,GLAT,Signif_Avg,CLASS1,5_PDF,RF_prob,RF_class,PDF_prob,PDF_class
4FGL J0000.3-7355,-11.760649,0.109999,0.030076,307.708984,-42.729538,8.492646,unas,0.126976,0.948705,bll+,0.575888,bll+
4FGL J0000.5+0743,-11.714305,0.109660,-0.349232,101.656479,-53.029457,5.681097,unas,0.320507,0.491909,fsrq+,0.350479,bll+
4FGL J0000.7+2530,-12.094175,0.251122,0.948690,108.775070,-35.959175,4.197268,unas,0.000004,0.956818,bll+,0.968732,bll+
4FGL J0001.2+4741,-11.863557,0.049796,0.009480,114.250198,-14.338059,5.523873,bcu,0.039535,0.903150,bll+,0.832919,bll+
4FGL J0001.2-0747,-11.107484,0.037899,0.014316,89.032722,-67.305008,24.497219,bll,0.515618,0.985132,bll+,0.515618,Gauss
...,...,...,...,...,...,...,...,...,...,...,...,...
4FGL J2359.3+2502,-11.877070,0.135214,0.473850,108.247383,-36.340054,5.558664,unas,0.004069,0.939121,bll+,0.854659,bll+
4FGL J2359.3+1444,-11.627869,0.222978,-0.689550,104.564713,-46.256252,7.791155,unas,0.629332,0.672280,fsrq+,0.629332,Gauss
4FGL J2359.7-5041,-11.952324,0.446554,-0.455627,322.128052,-64.472755,5.150436,bcu,0.645864,0.440625,fsrq+,0.645864,Gauss
4FGL J2359.9-3736,-11.756382,0.185941,0.285051,345.662842,-74.919624,11.475555,bcu,0.050086,0.809720,bll+,0.661594,bll+


In [116]:
# calculate confusion matrix
unas_mask = (out_df['CLASS1'] == 'unas') | (out_df['CLASS1'] == 'unk') #| (out_df['CLASS1'] == 'bcu')
assoc_mask = ~unas_mask
#unas_mask = (out_df['CLASS1'] == 'bcu')
bmask = 10
lat_mask = np.abs(out_df['GLAT']) < bmask
mask = unas_mask & lat_mask
classes1 = list(sum_df1.index)
classes2 = list(sum_df2.index)
if add_unc:
    classes1.append(unc_name)
    classes2.append(unc_name)
conf_df = pd.DataFrame(columns=classes2, index=classes1)

In [117]:
if add_unc:
    for mdl in methods:
        probs = out_df['%s_prob' % mdl]
        unc_mask = probs < pmin
        out_df.loc[unc_mask, '%s_class' % mdl] = unc_name
for cl1 in classes1:
    for cl2 in classes2:
        #print(cl1, cl2)
        #print(sum((out_df[mask]['RF_class'] == cl1)))
        res = sum((out_df[mask]['RF_class'] == cl1) & (out_df[mask]['PDF_class'] == cl2))
        conf_df.loc[cl1, cl2] = res


In [118]:
conf_df.index.name = 'Covar\Prior'

In [119]:
conf_df

,fsrq+,bll+,psr+,msp+,Gauss,uncert
Covar\Prior,,,,,,
fsrq+,9,0,0,0,21,2
bll+,5,75,0,0,0,13
psr+,37,5,19,9,175,21
msp+,0,0,2,80,176,16
uncert,79,21,0,37,338,35


In [120]:
txt = util.df2tex(conf_df, fmt='%.1f', index=True, index_name='')
print(txt)

\begin{tabular}{lllllll}
\hline
 & fsrq+ & bll+ & psr+ & msp+ & Gauss & uncert \\
Covar\Prior &  &  &  &  &  &  \\
\hline
fsrq+ & 9 & 0 & 0 & 0 & 21 & 2 \\
bll+ & 5 & 75 & 0 & 0 & 0 & 13 \\
psr+ & 37 & 5 & 19 & 9 & 175 & 21 \\
msp+ & 0 & 0 & 2 & 80 & 176 & 16 \\
uncert & 79 & 21 & 0 & 37 & 338 & 35 \\
\hline
\end{tabular}



In [121]:
sum(out_df['PDF_class'] == unc_name), sum(out_df['RF_class'] == unc_name)

(630, 1034)

In [122]:
sum(out_df.loc[mask, 'PDF_class'] == unc_name), sum(out_df.loc[mask, 'RF_class'] == unc_name)

(87, 510)

In [123]:
sum((out_df['PDF_class'][unas_mask] == 'Gauss') & (out_df['PDF_prob'][unas_mask] > 0.5))

913

In [124]:
gind

'5'

In [125]:
sum(mask)

1175

In [126]:
for cl in set(out_df['PDF_class']):
    print(cl, sum((out_df['PDF_class'][mask] == cl)))

psr+ 21
bll+ 101
fsrq+ 130
uncert 87
Gauss 710
msp+ 126


In [143]:
print('High lat')
print('Class, expected number, fration, number of candidates')
lat_mask = np.abs(out_df['GLAT']) > bmask
nunas = sum(unas_mask & lat_mask)

for i, clm in enumerate(['1_PDF', '2_PDF', '3_PDF', '4_PDF', 'Gauss_PDF']):
    cl = classes2[i]
    nclexp = sum((cat_df2[clm][unas_mask & lat_mask]))
    ncl = sum((out_df['PDF_class'][unas_mask & lat_mask] == cl))
    print('%s, %.1f, %.3f, %i' % (classes2[i], nclexp, nclexp/nunas, ncl))
print('total number of unas and unk sources: %i' % nunas)
print()
print('Low lat')
print('Class, expected number, fration, number of candidates')
lat_mask = np.abs(out_df['GLAT']) < bmask
nunas = sum(unas_mask & lat_mask)
for i, clm in enumerate(['1_PDF', '2_PDF', '3_PDF', '4_PDF', 'Gauss_PDF']):
    cl = classes2[i]
    nclexp = sum((cat_df2[clm][unas_mask & lat_mask]))
    ncl = sum((out_df['PDF_class'][unas_mask & lat_mask] == cl))
    print('%s, %.1f, %.3f, %i' % (classes2[i], nclexp, nclexp/nunas, ncl))
print('total number of unas and unk sources: %i' % sum(unas_mask & lat_mask))


High lat
Class, expected number, fration, number of candidates
fsrq+, 186.6, 0.186, 133
bll+, 396.5, 0.395, 386
psr+, 12.2, 0.012, 2
msp+, 194.1, 0.193, 142
Gauss, 214.6, 0.214, 203
total number of unas and unk sources: 1004

Low lat
Class, expected number, fration, number of candidates
fsrq+, 180.4, 0.153, 130
bll+, 111.8, 0.095, 101
psr+, 67.2, 0.057, 21
msp+, 191.7, 0.163, 126
Gauss, 624.0, 0.531, 710
total number of unas and unk sources: 1175


In [135]:
classes2

['fsrq+', 'bll+', 'psr+', 'msp+', 'Gauss', 'uncert']